In [1]:
import pycutestmgr as cute
import time
import pdb 
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
import scipy.sparse as spc
from ipsolver import parse_cutest_like_problem, ipsolver, parse_matlab_like_problem, check_derivative
cute.clearCache('HS71')

# Import Problem

In [2]:
problems = [("CORKSCRW", {"T": 50}),  # 1
            ("COSHFUN", {"M": 20}),  # 2 --- many iteractions
            ("DIXCHLNV", {"N":100}), # 3
            ("HAGER4", {"N": 1000}), # 4
            ("HIMMELBK", {}), # 5
            ("NGONE", {"HNS": 49}), # 6 --- many iteractions
            ("OPTCNTRL", {}), # 7
            ("OPTMASS", {"N": 200}),  # 8 --- OPT 4.6e-4
            ("ORTHREGF", {"NPTS": 20}),  # 9
            ("SVANBERG", {"N": 500}),  # 10
            ("READING1", {"N": 100}), # 11  --- Fails !!! Warning --- Take a look
            ("HS7", {}), # 12
            ("HS10", {}), # 13
            ("HS11", {}), # 14
            ("HS13", {}), # 15
            ("HS14", {}), # 16
            ("HS16", {}), # 17
            ("HS17", {}), # 18
            ("HS19", {}), # 19
            ("HS20", {}), # 20
            ("HS22", {}), # 21
            ("HS24", {}), # 22
            ("HS26", {}), # 23
            ("HS28", {}), # 24
            ("HS31", {}), # 25
            ("HS32", {}), # 26
            ("HS33", {}), # 27
            ("HS39", {}), # 28
            ("HS46", {}), # 29
            ("HS51", {}), # 30
            ("HS52", {}), # 31
            ("HS53", {}), # 32
            ("HS63", {}), # 33
            ("HS64", {}), # 34
            ("HS65", {}), # 35
            ("HS70", {}), # 36
            ("HS71", {}), # 37
            ("HS72", {}), # 38
            ("HS73", {}), # 39
            ("HS74", {}), # 40
            ("HS75", {}), # 41
            ("HS77", {}), # 42
            ("HS78", {}), # 43
            ("HS79", {}), # 44
            ("HS80", {}), # 45
            ("HS81", {}), # 46
            ("HS83", {}), # 47
            ("HS84", {}), # 48
            ("HS85", {}), # 49 --- Fails!
            ("HS86", {}), # 50
            ("HS93", {}), # 51
            ("HS95", {}), # 52
            ("HS96", {}), # 53
            ("HS97", {}), # 54 --- Fails!
            ("HS98", {}), # 55 --- Fails!
            ("HS99", {}), # 56 --- Fails!
            ("HS100", {}), # 57
            ("HS104", {}), # 58
            ("HS106", {}), # 60
            ("HS107", {}), # 61
            ("HS108", {}), # 62
            ("HS109", {}), # 63  --- Fails! Singular factor --- Take a look!!
            ("HS111", {}), # 64
            ("HS112", {}), # 65
            ("HS113", {}), # 66
            ("HS114", {}), # 67
            ("HS116", {}), # 68
            ("HS117", {}), # 69 
            ("HS118", {}), # 70
            ("HS119", {}), # 71 
            ]

In [3]:
number = 8
name = problems[number-1][0]
cute.clearCache(name)
cute.prepareProblem(name, sifParams=problems[number-1][1])
problem = cute.importProblem(name)
info = problem.getinfo()
x0 = info["x"]
n = info["n"]
m = info["m"]
print(n, m)
v0 = info["v"]
eq_list = info["equatn"]

def convert_to_inf(bounds):
    new_bounds = np.zeros_like(bounds)
    i = 0
    for b in bounds:
        if b >= 1e20:
            new_bounds[i] = np.inf
        elif b <= -1e20:
            new_bounds[i] = -np.inf
        else:
            new_bounds[i] = b
        i += 1
    return new_bounds

# Define upper and lower bound
c_lb = convert_to_inf(info["cl"])
c_ub = convert_to_inf(info["cu"])
lb = convert_to_inf(info["bl"])
ub = convert_to_inf(info["bu"])

# Define function and constraints
def fun(x):
    return problem.obj(x)[0]

def grad(x):
    _, g1 = problem.obj(x, True)
    return g1

def lagr_hess(x, v):
    def matvec(p):
        return problem.hprod(p, x, v)
    return spc.linalg.LinearOperator((n, n), matvec=matvec)


def constr(x):
    return problem.cons(x)

def jac(x):
    _, A1 = problem.cons(x, True)
    return spc.csc_matrix(A1)

# Make initial point feasible
if number == 59:
    x0[2] = x0[2] + 6
    x0[3] = x0[3] + 6

# Parse constraints
fun, grad, new_lagr_hess, n_total_ineq, constr_ineq, \
    jac_ineq, n_total_eq, constr_eq, jac_eq, box_constraints \
        = parse_cutest_like_problem(
            fun, grad, lagr_hess, n, constr, jac,
            c_lb, c_ub, lb, ub)
        
# Remove box contraints
if number not in (59, 65):
       box_constraints.fill(0)

(1210, 1005)


# Optimize Problem

In [4]:
x, info = ipsolver(fun, grad, new_lagr_hess, n_total_ineq, constr_ineq,
                jac_ineq, n_total_eq, constr_eq, jac_eq, x0, box_constraints)

# Display Results

In [5]:
print('f = '+ str(info["fun"]))
print('optimality = ' + str(info["opt"]))
print('c violation = ' + str(info["constr_violation"]))
print('niter = ' + str(info["niter"]))
#print('total time = ' + str(total_time))

f = -0.0327169023548
optimality = 0.000520989535076
c violation = 2.20472516866e-06
niter = 1139
